In [1]:
import torch
import argparse
import torchio as tio
import numpy as np
import nibabel as nib
from box import Box
import os.path as path

import sys, os, json
sys.path.append(os.path.join(sys.path[0], '../'))
from dataloader.patch_dataloader import patch_dataloader
from dataloader.data_utils import load_onefold_dataset, load_all
from network.VNet_model import VNetModel
from network.WatNet_model import WatNet3DModel, WatNet2DModel


/home/dual4090/anaconda3/envs/3t7t/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/dual4090/anaconda3/envs/3t7t/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit 

In [2]:
def load_params(param_dir):
    with open(param_dir, 'r') as json_file:
        params = json.load(json_file)
    return Box(params)

In [3]:
param_path = '../../config/params_VNet_eval.json'
params = load_params(param_path)

In [4]:
print(params)

{'fp': {'subjects': '../../data/subjects.txt', 'filepaths': ['../../data/original'], 'postfixes': ['.nii.gz'], 'eval_modes': ['org', 'down2', 'down4'], 'ckpt_dir': '../../ckpts/tensorlog/VNet_final_ckpts', 'leave_out_subjects': []}, 'data': {'batch_size': 40, 'patch_size': 64, 'patch_overlap': 8}, 'training': {'num_workers': 32, 'max_queue_length': 1000, 'matmul_precision': 'medium', 'precision': '16-mixed'}, 'checkpoint': {'ckpt_dir': '../../ckpts/tensorlog'}}


In [5]:
empty_params = Box()

In [6]:
empty_params.faa = params.checkpoint

In [7]:
empty_params

Box({'faa': {'ckpt_dir': '../../ckpts/tensorlog'}})

# Loading args

In [8]:
ckpt_path = r'/home/dual4090/lab/github/synth7T-MICCAI/ckpts/tensorlog/VNet_final_ckpts/MAEloss_finalmodel/Synthetic_7T_MRI_VNet_weights.ckpt'

In [9]:
if not os.path.exists(ckpt_path):
    raise ValueError('Checkpoint path does not exist')

if not ckpt_path.endswith('.ckpt'):
    raise ValueError('Please provide a .ckpt file')

In [11]:
input_path =r'/home/dual4090/lab/github/3T-7T_registration/data/original/3T/01.nii.gz'

if not os.path.exists(input_path):
    raise ValueError('Input path does not exist')

if os.path.isdir(input_path):
    # input is dir, get all .nii.gz files in input dir
    input_files = [f for f in os.listdir(input_path) if f.endswith('.nii.gz')]
    if not input_files:
        raise ValueError('No .nii.gz files found in input directory')
    
elif os.path.isfile(input_path):
    # input is file, check if it is .nii.gz
    if not input_path.endswith('.nii.gz'):
        raise ValueError('Input file is not a .nii.gz file')
    input_files = [input_path]

['/home/dual4090/lab/github/3T-7T_registration/data/original/3T/01.nii.gz']

# Functions

In [39]:
path.exists('')

False

In [ ]:
def check_filePaths(file_paths):
    # make sure all file paths in list is a nifti file

In [ ]:
def load_MRIs(fps_input, fps_mask = None):
    
    if fps_mask is not None:
        flag_mask = True

    subjects = []
    for i in range(len(fns)):
        # normalize value between 0,1
        # outlier removal done during preprocessing (5%-95% percentile kept)
        rescale1 = tio.RescaleIntensity(out_min_max=(0, 1), percentiles=(0, 100))

        subject_dict = {}
        if flag3:
            subject_dict['t1_3T'] = rescale1(tio.ScalarImage(path.join(fp3, str(fns[i]) + postfix)))
        if flag_mask:
            subject_dict['mask'] = tio.LabelMap(path.join(fpm, str(fns[i]) + postfix))

### what happens if there is no mask?

        subject = tio.Subject(id=fns[i], **subject_dict)

        subjects.append(subject)
    # print('Dataset size:', len(subjects), 'subjects')
    return subjects


In [36]:
def load_all_from_paths(filePaths):
    #filePaths is a list of file paths
    subjects =[]
    for i in range(len(filePaths)):
        subjects.extend(load_MRIs(filePaths[i]))

    tio_dataset = tio.SubjectsDataset(subjects)
    print('total number of subjects: ', len(subjects))
    return tio_dataset